This version improve the matching code from the answer to fuzzy

In [1]:
# Login to Weights & Biases for experiment tracking
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: qtra0027 (ailecs-lab-students) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
# Initialize a new Weights & Biases run for experiment tracking
run = wandb.init(
    project='Using Gemma3_4b to classify illicit content on online marketplace (binary classification)_ver2', 
    job_type="training", 
    anonymous="allow"
)

In [3]:
import numpy as np
import pandas as pd
import os
import bitsandbytes as bnb
import evaluate
import torch
import torch.nn as nn
import transformers
from datasets import Dataset, DatasetDict
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import (GemmaForSequenceClassification, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          DataCollatorWithPadding)
from sklearn.metrics import (accuracy_score, 
                            precision_score, 
                            recall_score, 
                            f1_score, 
                            classification_report, 
                            confusion_matrix)
from sklearn.model_selection import train_test_split
from transformers import pipeline, Trainer

In [4]:
# Load JSONL file
file_path = "DUTA10K_final.jsonl"
df = pd.read_json(file_path, lines=True)

# List all unique categories
unique_categories = df["category"].dropna().unique()

# Print the categories
print("✅ Unique categories found:")
for cat in sorted(unique_categories):
    print("-", cat)

# Get category counts
print("\n📊 Category distribution:")
print(df["category"].value_counts())

✅ Unique categories found:
- Art_Music
- Casino_Gambling
- Counterfeit Credit-Cards
- Counterfeit Money
- Counterfeit Personal-Identification_Driving-Licence
- Counterfeit Personal-Identification_ID
- Counterfeit Personal-Identification_Passport
- Cryptocurrency
- Cryptolocker
- Drugs_Illegal
- Drugs_Legal
- Forum_Illegal
- Forum_Legal
- Fraud
- Hacking
- Hosting_Directory
- Hosting_File-sharing
- Hosting_Folders
- Hosting_Search-Engine
- Hosting_Server
- Hosting_Software
- Human-Trafficking
- Leaked-Data
- Library_Books
- Marketplace_Illegal
- Marketplace_Legal
- Personal
- Politics
- Porno_Child-pornography
- Porno_General-pornography
- Religion
- Services_Illegal
- Services_Legal
- Social-Network_Blog
- Social-Network_Chat
- Social-Network_Email
- Social-Network_News
- Violence_Hate
- Violence_Hitman
- Violence_Weapons

📊 Category distribution:
category
Hosting_Server                                         760
Cryptocurrency                                         577
Personal     

In [5]:
# Define split sizes and split the DataFrame into training, evaluation, and test sets
n = len(df)
train_end = int(0.8 * n)
eval_end  = train_end + int(0.1 * n)

df_train = df.iloc[:train_end]
df_eval  = df.iloc[train_end:eval_end]
df_test  = df.iloc[eval_end:]

print(f"Train={len(df_train)} | Eval={len(df_eval)} | Test={len(df_test)}")

Train=3342 | Eval=417 | Test=419


In [6]:
# Create a DatasetDict from the pandas DataFrames
ds = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "eval":  Dataset.from_pandas(df_eval),
    "test":  Dataset.from_pandas(df_test),
})

In [7]:
# Load the base model tokenizer
base_model_name = "google/gemma-3-4b-it"

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)

MAX_LEN = 10000  # or 1,024 if you have the headroom

# Preprocess function to tokenize the text data
def preprocess(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding="max_length"   # pad shorter examples up to exactly MAX_LEN
    )

# Apply preprocessing to the datasets
tokenized = ds.map(preprocess, batched=True)

Map:   0%|          | 0/3342 [00:00<?, ? examples/s]

Map:   0%|          | 0/417 [00:00<?, ? examples/s]

Map:   0%|          | 0/419 [00:00<?, ? examples/s]

In [8]:
# Prepare Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
# Metrics fuction
metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

In [10]:
# Load GEMMA‑3 for sequence classification
model = GemmaForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=2,
    id2label={0:"non‑illicit", 1:"illicit"}, # Mapping from ID to label string
    label2id={"non‑illicit":0, "illicit":1}, # Mapping from label string to ID
    torch_dtype=torch.float16, # Use float16 for reduced memory usage
    device_map="auto", # Automatically maps model to available devices
    trust_remote_code=True, # Allows loading custom code from the model's repository
)

# Prepare the model for k-bit training (LoRA compatible)
model = prepare_model_for_kbit_training(model)
model.gradient_checkpointing_enable() # Enable gradient checkpointing to save memory during training

You are using a model of type gemma3 to instantiate a model of type gemma. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at google/gemma-3-4b-it and are newly initialized: ['model.embed_tokens.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.10.input_layernorm.weight', 'model.layers.10.mlp.down_proj.weig

In [11]:
from transformers.pytorch_utils import Conv1D

# Define supported module types for LoRA adaptation
SUPPORTED = (nn.Linear, nn.Embedding, nn.Conv1d, nn.Conv2d, nn.Conv3d, Conv1D, nn.MultiheadAttention) # Expanded list from original

target_modules = [
    name.split(".")[-1]
    for name, module in model.named_modules()
    if isinstance(module, SUPPORTED)
]
print("LoRA will target:", target_modules)

LoRA will target: ['embed_tokens', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj', 'k_proj', 'v_proj', 'o_proj', '

In [12]:
# LoRA adapter configuration
lora_config = LoraConfig(
    r=64, # LoRA rank
    lora_alpha=32, # LoRA scaling factor
    target_modules=target_modules, # Modules to apply LoRA to
    lora_dropout=0.1, # Dropout probability for LoRA layers
    bias="none", # Do not apply bias to LoRA weights
    task_type="SEQ_CLS", # Sequence Classification task
)

# Get the PEFT (Parameter-Efficient Fine-Tuning) model
model = get_peft_model(model, lora_config) 
model.gradient_checkpointing_enable() # Re-enable gradient checkpointing after wrapping with PEFT model
model.print_trainable_parameters() # Print the number of trainable parameters

# Training arguments configuration
training_args = TrainingArguments(
    output_dir="gemma_4b_binary_ver2", # Output directory for checkpoints and logs
    per_device_train_batch_size=1, # Batch size per GPU for training
    per_device_eval_batch_size=1, # Batch size per GPU for evaluation
    gradient_accumulation_steps=8, # Number of updates steps to accumulate before performing a backward/update pass
    learning_rate=2e-5, # Initial learning rate for AdamW optimizer
    num_train_epochs=8, # Total number of training epochs
    fp16=True, # Enable mixed precision training
    eval_strategy="epoch", # Evaluation is done at the end of each epoch
    save_strategy="epoch", # Model is saved at the end of each epoch
    load_best_model_at_end=True, # Load the best model at the end of training
    metric_for_best_model="accuracy", # Metric to use to compare models
    report_to=["wandb"], # Report metrics to Weights & Biases
)

# Initialize the Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["eval"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/home/ubuntu/.local/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:529: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.embed_tokens'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(
/tmp/ipykernel_2306959/2588455016.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 216,602,624 || all params: 8,754,289,664 || trainable%: 2.4742


In [13]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.822356,0.729017,0.331361,0.294737,0.378378
2,0.785800,0.764468,0.788969,0.405405,0.405405,0.405405
3,0.143200,0.876286,0.786571,0.350365,0.380952,0.324324
4,0.047700,1.222065,0.810552,0.347107,0.446809,0.283784
5,0.004400,1.317229,0.815348,0.363636,0.468085,0.297297
6,0.000000,1.326602,0.810552,0.347107,0.446809,0.283784
7,0.000000,1.327025,0.810552,0.347107,0.446809,0.283784


/home/ubuntu/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:221: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ubuntu/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:221: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ubuntu/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:221: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/ubuntu/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:221: UserWarning: Setting `save_embedding_layer

TrainOutput(global_step=3336, training_loss=0.1470525722986431, metrics={'train_runtime': 239428.8868, 'train_samples_per_second': 0.112, 'train_steps_per_second': 0.014, 'total_flos': 1.275207811977216e+19, 'train_loss': 0.1470525722986431, 'epoch': 7.981448234590066})

In [14]:
# Save trained model and tokenizer
trainer.save_model("gemma_4b_binary_ver2")
tokenizer.save_pretrained("gemma_4b_binary_ver2")

/home/ubuntu/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:221: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('gemma_4b_binary_ver2/tokenizer_config.json',
 'gemma_4b_binary_ver2/special_tokens_map.json',
 'gemma_4b_binary_ver2/chat_template.jinja',
 'gemma_4b_binary_ver2/tokenizer.model',
 'gemma_4b_binary_ver2/added_tokens.json',
 'gemma_4b_binary_ver2/tokenizer.json')

In [15]:
# Make sure model & tokenizer are on the right device
device = next(model.parameters()).device
model.to(device)
model.eval()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GemmaForSequenceClassification(
      (model): GemmaModel(
        (embed_tokens): lora.Embedding(
          (base_layer): Embedding(256000, 3072, padding_idx=0)
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.1, inplace=False)
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 64x256000 (cuda:0)])
          (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 3072x64 (cuda:0)])
          (lora_magnitude_vector): ModuleDict()
        )
        (layers): ModuleList(
          (0-27): 28 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=4096, bias=False)
                (lora_dropout): Modul

In [16]:
# Function to make predictions on a single text input
def predict(text: str):
    # tokenize + move to device
    inputs = tokenizer(
        text,
        return_tensors="pt", # Return PyTorch tensors
        truncation=True, # Truncate if longer than max_length
        max_length=512, # Max length for inference
        padding="max_length" # Pad to max_length
    ).to(device)
    
    # Perform forward pass without gradient calculation
    with torch.no_grad():
        logits = model(**inputs).logits  # Get raw logits from the model
        
    # Apply softmax to get probabilities and convert to numpy array
    probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]
    
    # Get the index of the highest probability
    idx   = int(np.argmax(probs))
    return {
        "label":    id2label[idx], # Predicted label string
        "score":    float(probs[idx]), # Score of the predicted label
        "all_probs": { id2label[i]: float(probs[i]) for i in range(len(probs)) }
    }

In [17]:
# Get predictions on the test set using the trainer
preds_output = trainer.predict(tokenized["test"])
y_true = preds_output.label_ids
y_pred = np.argmax(preds_output.predictions, axis=-1)

In [18]:
# Compute & print metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec  = recall_score(y_true, y_pred, zero_division=0)
f1   = f1_score(y_true, y_pred, zero_division=0)

In [19]:
print(f"Test Accuracy:  {acc:.4f}")
print(f"Test Precision: {prec:.4f}")
print(f"Test Recall:    {rec:.4f}")
print(f"Test F1:        {f1:.4f}")

# Full classification report + confusion matrix
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["non‑illicit","illicit"], zero_division=0))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

Test Accuracy:  0.8258
Test Precision: 0.5873
Test Recall:    0.4405
Test F1:        0.5034

Classification Report:
              precision    recall  f1-score   support

 non‑illicit       0.87      0.92      0.89       335
     illicit       0.59      0.44      0.50        84

    accuracy                           0.83       419
   macro avg       0.73      0.68      0.70       419
weighted avg       0.81      0.83      0.82       419

Confusion Matrix:
[[309  26]
 [ 47  37]]


In [20]:
# Finish the Weights & Biases run
wandb.finish()
model.config.use_cache = True

eval/accuracy,▁▆▆█████
eval/f1,▁█▃▂▄▂▂▂
eval/loss,▂▁▂▇████
eval/precision,▁▅▄▇█▇▇▇
eval/recall,▆█▃▁▂▁▁▁
eval/runtime,▁▄▆█▃▂▂▂
eval/samples_per_second,██▁▁████
eval/steps_per_second,██▁▁████
test/accuracy,▁
test/f1,▁
test/loss,▁


In [ ]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
import torch
import os

# 🔐 Login to HuggingFace
from getpass import getpass
hf_token = getpass("Enter your HuggingFace token: ")
login(token=hf_token)

In [ ]:
# 🧠 Base and fine-tuned model paths
base_model = "gemma_4b_binary_ver2"  # You used this in your training code
fine_tuned_model = "gemma_4b_binary_ver2"  # Your output dir from training

# 🔁 Reload tokenizer and base model
print("🔄 Loading base tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = GemmaForSequenceClassification.from_pretrained(
    base_model,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
# 📎 Merge adapter
print("🔗 Merging LoRA adapter with base model...")
model = PeftModel.from_pretrained(base_model_reload, fine_tuned_model)
model = model.merge_and_unload()

# 💾 Save locally and push to HF Hub
model_dir = "gemma_4b_binary_ver2"
os.makedirs(model_dir, exist_ok=True)
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

# ☁️ Push to Hugging Face
model.push_to_hub(model_dir, use_temp_dir=False)
tokenizer.push_to_hub(model_dir, use_temp_dir=False)